In [ ]:
import sys
sys.path.append('..')
from augmentations import *
from dataset import *
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision.transforms import v2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# settings

In [ ]:
# Hyperparameter Setting
CFG = {
    "ROOT": '../train_final', # 학습 데이터 경로
    "WORK_DIR": './final_kold/final_fold4',
    # retraining 설정
    "START_FROM": None, # 만약 None이 아닌 .pth파일 경로 입력하면 해당 checkpoint를 load해서 시작
    "GROUP_PATH": None, # 만약 None이 아닌 group.json의 경로르 입력하면 해당 class들만 활용하여 train을 진행함
    
    # wrong example을 뽑을 threshold 조건. threshold 이하인 confidence를 가지는 케이스를 저장.
    "WRONG_THRESHOLD": 0.7,
    "GROUP_JSON_START_EPOCH": 5, # work_dir에 해당 에폭부터의 wrong_examples를 통합한 json파일을 저장하게됩니다.

    # 해당 augmentation들은 선택된 것들 중 랜덤하게 '1개'만 적용이 됩니다(배치마다 랜덤하게 1개 선택)
    "ALL_AUGMENTATIONS": ["CUTMIX", "MIXUP", "MOSAIC", "CUTOUT", "SALIENCYMIX"], # 여기에 정의되어 있는 것 중 True만 실제 적용. 
    "NONE_AUGMENTATION_LIST": [],
    "CUTMIX": {
        'enable': False,
        'params':{'alpha':1.0} # alpha값 float로 정의 안하면 오류남
    },
    "SALIENCYMIX": {
        'enable': False,
        'params':{'alpha':1.0, 'num_candidates':9}
    },
    "MIXUP": {
        'enable': True,
        'params':{'alpha':1.0} # alpha값 float로 정의 안하면 오류남
    },
    "MOSAIC": {
        'enable': False,
        'params':{
            'p': 1.0,
            'grid_size': 2,
            'use_saliency': False
        }
    },
    "CUTOUT": {
        'enable': False,
        'params':{
            'mask_size': 32
        }
    },

    # 기타 설정값들
    'IMG_SIZE': 600, # Number or Tuple(Height, Width)
    'BATCH_SIZE': 32, # 학습 시 배치 크기
    'EPOCHS': 35,
    'SEED' : 42,
    'MODEL_NAME': 'convnext_large_mlp.clip_laion2b_augreg_ft_in1k_384',
    'N_FOLDS': 5,
    'EARLY_STOPPING_PATIENCE': 5,
    'RUN_SINGLE_FOLD': True,  # True로 설정 시 특정 폴드만 실행
    'TARGET_FOLD': 4,          # RUN_SINGLE_FOLD가 True일 때 실행할 폴드 번호 (1-based)
    

    # 새롭게 추가된 logging파트. class의 경우 무조건 풀경로로 적어야합니다. nn.CrossEntropyLoss 처럼 적으면 오류남
    'LOSS': {
        'class': 'torch.nn.CrossEntropyLoss',
        'params': {}   
    },
    'OPTIMIZER': {
        'class': 'torch.optim.AdamW',
        'params': {
            'lr': 2e-05,
            'weight_decay': 0.01
        }
    },
    'SCHEDULER': {
        'class': 'torch.optim.lr_scheduler.CosineAnnealingLR',
        'params': {
            'T_max': 35,
            'eta_min': 2e-08
        }
    },
}

CFG['IMG_SIZE'] = CFG['IMG_SIZE'] if isinstance(CFG['IMG_SIZE'], tuple) else (CFG['IMG_SIZE'], CFG['IMG_SIZE'])
# --- Albumentations 기반 이미지 변환 정의 ---
train_transform = A.Compose([
    CustomCropTransformConsiderRatio(p=0.5, consider_ratio=True),
    A.Resize(CFG['IMG_SIZE'][0], CFG['IMG_SIZE'][1]),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.Affine(translate_percent=(0.1, 0.1), scale=(0.9, 1.1), shear=10, rotate=0, p=0.5),
    A.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0]),
    ToTensorV2()
], seed=CFG['SEED'])

# load data

In [ ]:
train_root = CFG['ROOT']
initial_dataset = InitialCustomImageDataset(train_root)
if not initial_dataset.samples:
    raise ValueError(f"No images found in {train_root}. Please check the path and data structure.")
print(f"총 학습 이미지 수 (K-Fold 대상): {len(initial_dataset.samples)}")

all_samples = initial_dataset.samples
targets = [s[1] for s in all_samples]
class_names = initial_dataset.classes
num_classes = len(class_names)
print(f"클래스: {class_names} (총 {num_classes}개)")

In [ ]:
train_dataset = FoldSpecificDataset(all_samples, image_size = CFG['IMG_SIZE'], transform=train_transform, is_train=False)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)

# check images

In [ ]:
all_mix_augmentations = RandomMixAugmentation(CFG, num_classes=num_classes)

In [ ]:
def show_transformed_vs_original(dataloader, num_samples=10, device='cpu'):
    """
    무작위 배치 하나에서 num_samples만큼의 이미지에 대해
    원본 이미지와 transform된 이미지를 나란히 시각화.

    Args:
        dataloader: PyTorch DataLoader
        num_samples: 시각화할 이미지 수
        device: 'cuda' 또는 'cpu'
    """
    # dataloader에서 배치 하나만 추출
    for images, labels, img_paths in dataloader:
        images, labels = images.to(device), labels.to(device)
        print(labels)
        num_samples = min(num_samples, images.size(0))
        
        images, labels = all_mix_augmentations.forward(images, labels)

        fig, axs = plt.subplots(num_samples, 2, figsize=(10, 5 * num_samples))

        if num_samples == 1:
            axs = [axs]

        for i in range(num_samples):
            # transform된 이미지 (Tensor → numpy)
            print(img_paths[i])
            transformed_img = images[i].cpu()
            transformed_np = transformed_img.permute(1, 2, 0).numpy()

            # 원본 이미지 (img_path에서 로드)
            original_img = Image.open(img_paths[i]).convert('RGB')

            axs[i][0].imshow(original_img)
            # axs[i][0].set_title(f"Original: {img_paths[i].split('/')[-1]}")
            axs[i][0].set_title(f"Original")
            axs[i][0].axis('off')

            axs[i][1].imshow(transformed_np)
            axs[i][1].set_title("Transformed")
            axs[i][1].axis('off')

        plt.tight_layout()
        plt.show()
        break

In [ ]:
show_transformed_vs_original(train_loader)